In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt
from idaes.apps.uncertainty_propagation.sens import get_dsdp

    'pyomo.common.sorting.sorted_robust'.  Please update your import.
    (deprecated in 6.1) (called from <frozen importlib._bootstrap>:228)


## Define fixed bed model 

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y):
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}}
    return dv_dict_overall

In [4]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
initialize_point = './priors/20210916_feed313_bath293.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#initialize_point = './priors/20210916_feed313_bath293_5e3.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [5]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=293.15, y=0.4, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [6]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
#dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}
dv_pass = {'temp_feed': None, 'yfeed': None}

measurement = ['FCO2', 'temp']
measurement_extra = {'FCO2': [19], 'temp': [10, 19]}

t_measure = exp_time_


In [7]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


In [32]:
fim_init = [[0.0029331 , 0.01236752],
 [0.01236752 , 0.71783208]]

#f2 = open('jac_initial', 'rb')
#jac_init = pickle.load(f2)
#f2.close()

jac_init = jac_init_1.copy()
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

[[0.0541581  0.        ]
 [0.22835955 0.8158946 ]]


In [24]:
import os

scores = {} # scores is an empty dict already
target='jac_initial'
if os.path.getsize(target) > 0:      
    with open(target, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()
        
print(scores)

{}


## Compute FIM

In [ ]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
##sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(293.15, 293.15, 0.4)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output='store_output_fco2',
                                formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.design_values)

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
print(doe_object.dsdp)

In [ ]:
import pickle

with open('store_output_botht', 'rb') as f:
    data = pickle.load(f)
    f.close()
print(data)

In [ ]:
doe_object.models[0]

In [ ]:
breakthrough_modify2(doe_object.models[2])

In [ ]:
def breakthrough_modify2(m, file=None, source="lab"):
    '''
    Draw breakthrough curve of the Pyomo model
    
    Arguments:
        m: Pyomo model
        file: when source = 'computer', this is where the computer experimental data is stored. Otherwise, its default is None
        source: if computer, it is plotting the computer experiment data. If lab, its comparing the experiments from the lab
    
    Return: None 
    
    Other: plot
    
    '''
    
    
    # unit: mol/m3
    outlet_den, _, _, _ = extract3d(m, m.C, 'CO2')
    
    # unit: cm/s
    outlet_vel, _, _, _ = extract2d(m, m.v)
    
    outlet_n2, _, _, _ = extract3d(m, m.C, 'N2')
    
    model_temp, _,_,_ = extract2d(m,m.temp)
    
    
    T = []
    for j in m.t:
        T.append(value(j))
        t_final = T[-1]
        
    
    if source == "lab":
        break_wvu = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/breakthrough_wvu.csv')
    
        exp = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/co2_breakthrough.csv')
        
        data_c1 = exp['FCO2']
        data_c = np.asarray(data_c1)
        data_t1 = exp['time']  
        data_t = np.asarray(data_t1*60+10)
        new_data = np.interp(T, data_t, data_c)
    
        fco2 = []
        for t in m.t:
            fco2.append(value(m.FCO2[0,19,t]))
        
        yco2 = []
        for i in range(len(fco2)):
            yco2.append(fco2[i]/fco2[-1])
            
        # real value of FCO2. for verification
        comp = outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1])
    
        plt.plot(exp['time']*60, exp['FCO2'],'b.', color='r', label='Experimental data')
        #plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.plot(T, yco2, label='Model prediction')
        
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
    elif source == "computer": 
        
        sol = pd.read_csv(file)
    
        #for i in range(len(time_exp)):    
        #    yco2_exp[time_exp[i]] = sol['FCO2'][i]
        #    temp_mid_exp[time_exp[i]] = sol['temp_mid'][i]
        #    temp_end_exp[time_exp[i]] = sol['temp_end'][i]
        
        exp_fco2 = np.asarray(sol['FCO2'].values.tolist())
        exp_temp_mid = np.asarray(sol['temp_mid'].values.tolist())
        exp_temp_end = np.asarray(sol['temp_end'].values.tolist())
        
        plt.plot(T, exp_fco2/exp_fco2[-1], label = 'Experimental data')
        plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, exp_temp_mid, label='Experimental data of middle T')
        plt.plot(T, model_temp[10,:], label='Model prediction of middle T')
        plt.plot(T, exp_temp_end, label='Experimental data of end T')
        plt.plot(T, model_temp[19,:], label='Model prediction of end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.title('Temperature model prediction and experimental data')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()

## Gradient-based optimization 

In [33]:
# feed, bath, y
exp1 = generate_exp(313.15, 293.15, 0.15)



In [35]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=l_init, jac_initial=jac_init, fim_initial=fim_init)

Sensitivity information is scaled by its corresponding parameter nominal value.
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is ./priors/20210916_feed313_bath293.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  313.15
yfeed is fixed at  0.15
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-ps


Number of Iterations....: 63

                                   (scaled)                 (unscaled)
Objective...............:   6.2386509881506607e+00    6.2386509881506607e+00
Dual infeasibility......:   1.4166397477310966e-06    1.4166397477310966e-06
Constraint violation....:   1.2228640409311931e-11    3.4924596548080450e-10
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   1.2228640409311931e-11    1.4166397477310966e-06


Number of objective function evaluations             = 70
Number of objective gradient evaluations             = 5
Number of equality constraint evaluations            = 70
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 65
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 63
Total CPU secs in IPOPT (w/o function evaluations)   =     47.223
Total CPU secs in NLP function evaluations

   1  6.2382265e+00 7.28e+01 3.79e+01  -1.0 4.08e+02    -  4.77e-03 1.78e-03h  1
   2  6.2382821e+00 7.28e+01 1.72e+02  -1.0 1.52e+03    -  8.40e-04 1.77e-05h  1
Reallocating memory for MA57: lfact (12878142)
   3r 6.2382821e+00 7.28e+01 9.99e+02   1.9 0.00e+00    -  0.00e+00 8.90e-08R  2
   4r 6.0263433e+00 6.63e+01 9.97e+02   1.9 7.18e+04    -  1.59e-03 1.26e-04f  1
   5r 6.0263433e+00 6.63e+01 9.99e+02   1.8 0.00e+00    -  0.00e+00 2.92e-07R  6
   6r 5.8634867e+00 6.30e+01 9.99e+02   1.8 6.26e+04    -  3.40e-04 1.21e-04f  1
   7  5.8640710e+00 6.29e+01 3.16e+01  -1.0 3.30e+02    -  9.59e-04 1.42e-03h  1
   8  5.8640919e+00 6.29e+01 2.27e+02  -1.0 7.43e+02    -  1.92e-03 1.41e-05h  1
   9r 5.8640919e+00 6.29e+01 9.99e+02   1.8 0.00e+00    -  0.00e+00 7.30e-08R  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10r 5.7334207e+00 6.03e+01 9.99e+02   1.8 5.46e+04    -  1.54e-04 1.18e-04f  1
  11r 5.7027469e+00 5.76e+01 9.98e+02   1.1 4.36e+04    -  6.3

  89 -3.9303580e+00 5.65e+04 4.19e+05  -1.0 2.72e+02    -  1.25e-04 1.70e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90 -4.4836337e+00 5.39e+04 4.44e+05  -1.0 2.79e+02    -  9.79e-04 1.94e-03f  1
  91 -4.4843917e+00 5.39e+04 4.44e+05  -1.0 2.80e+02    -  2.53e-03 1.97e-05h  1
  92 -4.4269415e+00 4.80e+04 3.78e+10  -1.0 3.06e+00  11.0 1.27e-02 1.14e-01h  1
  93r-4.4269415e+00 4.80e+04 9.99e+02   3.7 0.00e+00    -  0.00e+00 1.17e-07R  2
  94r-4.4731970e+00 1.98e+04 1.39e+05   3.7 5.02e+06    -  5.25e-02 7.99e-04f  1
  95 -4.4673882e+00 1.98e+04 3.55e+00  -1.0 1.94e+01    -  1.68e-03 1.12e-03h  1
  96 -4.4672296e+00 1.98e+04 4.99e+01  -1.0 9.35e+01    -  6.01e-04 2.98e-05h  1
  97 -4.4629794e+00 1.98e+04 4.72e+01  -1.0 2.15e+02    -  2.90e-04 9.92e-04h  1
  98r-4.4629794e+00 1.98e+04 9.99e+02   3.3 0.00e+00    -  0.00e+00 3.12e-07R  6
  99r-4.4388942e+00 6.49e+03 8.43e+04   3.3 2.07e+06    -  4.31e-02 9.91e-04f  1
iter    objective    inf_pr 

 195  1.5931545e+00 2.96e-01 4.86e+01  -1.0 3.55e+00    -  4.69e-01 4.63e-03h  7
 196  1.6019471e+00 2.95e-01 6.05e+01  -1.0 4.34e+00    -  5.73e-01 4.62e-03h  7
 197  1.6040463e+00 2.94e-01 6.64e+01  -1.0 6.59e+00    -  9.59e-02 1.15e-03h  9
 198  1.6061191e+00 2.94e-01 1.24e+02  -1.0 7.02e+00    -  5.91e-01 1.15e-03h  9
 199  1.8631653e+00 1.14e+01 6.91e+03  -1.0 1.21e+01    -  4.50e-02 1.46e-01f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200  2.0721487e+00 1.64e+01 1.03e+04  -1.0 1.30e+01    -  1.59e-01 1.29e-01f  2
 201  2.5125539e+00 4.58e+00 2.17e+04  -1.0 9.64e+00    -  2.48e-01 2.36e-01F  1
 202  2.5290806e+00 4.53e+00 2.18e+04  -1.0 9.99e+00    -  2.23e-01 7.53e-03f  1
 203  2.5289962e+00 4.53e+00 6.90e+06  -1.0 1.87e+02    -  8.71e-02 8.30e-05h  1
 204r 2.5289962e+00 4.53e+00 1.00e+03   0.7 0.00e+00    -  0.00e+00 2.60e-07R 12
 205r 2.5145963e+00 1.12e+00 1.88e+03   0.7 4.53e+03    -  3.43e-01 9.90e-04f  1
 206  2.5147261e+00 1.12e+00

 284 -2.4898130e-01 2.34e-01 3.67e+04  -1.0 1.66e+00    -  3.63e-01 1.75e-03h  1
 285 -2.4894035e-01 2.34e-01 1.16e+09  -1.0 1.74e+00    -  5.56e-01 1.76e-05h  1
 286r-2.4894035e-01 2.34e-01 1.00e+03  -0.6 0.00e+00    -  0.00e+00 8.81e-08R  2
 287r-2.3965703e-01 7.71e-02 9.96e+02  -0.6 2.31e+02    -  5.04e-03 1.00e-03f  1
 288 -2.3877329e-01 7.71e-02 1.67e+05  -1.0 1.54e+00    -  3.91e-01 4.25e-04h  1
 289r-2.3877329e-01 7.71e-02 9.99e+02  -1.0 0.00e+00    -  0.00e+00 2.66e-07R  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290r-2.2399010e-01 1.27e-01 1.01e+03  -1.0 6.87e+01    -  1.16e-03 1.31e-03f  1
 291r-2.1360278e-01 1.62e-01 9.95e+02  -1.0 3.07e+01    -  3.47e-03 2.58e-03f  1
 292r-2.0965860e-01 1.68e-01 9.80e+02  -1.0 7.48e+00    -  1.45e-02 6.21e-03f  1
 293r-2.0992870e-01 1.71e-01 8.90e+02  -1.0 2.17e+00    -  9.98e-02 1.67e-02f  1
 294r-2.1664227e-01 1.97e-01 6.69e+02  -1.0 1.85e+00    -  3.00e-01 1.13e-01f  1
 295r-2.4484199e-01 6.56e-01

 376  1.5091609e+00 1.47e+01 3.39e+12  -1.0 2.65e-03  15.3 1.00e+00 1.00e+00h  1
 377  1.5091661e+00 5.95e+00 3.22e+12  -1.0 4.62e-03  14.8 1.00e+00 1.00e+00h  1
 378  1.5091680e+00 3.84e+00 4.20e+12  -1.0 2.44e-03  15.2 1.00e+00 1.00e+00h  1
 379  1.5091685e+00 3.54e+00 2.69e+12  -1.0 2.96e-02  14.8 1.00e+00 8.79e-02H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380  1.5091854e+00 2.08e+00 1.27e+12  -1.0 4.73e-03  14.3 1.00e+00 1.00e+00h  1
 381  1.5092351e+00 2.08e+00 5.09e+11  -1.0 5.22e-03  13.8 3.71e-01 1.00e+00h  1
 382  1.5093803e+00 1.91e+00 1.69e+12  -1.0 1.57e-02  13.3 1.00e+00 1.00e+00h  1
 383  1.5093869e+00 1.34e+01 1.07e+13  -1.0 1.34e-02  14.7 5.55e-01 1.00e+00h  1
 384  1.5093893e+00 4.22e+01 7.29e+11  -1.0 7.79e-03  15.1 1.00e+00 1.00e+00h  1
 385  1.5093929e+00 2.59e+01 1.44e+12  -1.0 7.28e-03  14.6 1.00e+00 5.00e-01h  2
 386  1.5094137e+00 8.20e+00 9.00e+11  -1.0 6.67e-03  14.1 1.00e+00 1.00e+00h  1
 387  1.5094744e+00 1.93e+00

 469  3.8790937e-01 1.29e-01 1.49e+06  -1.0 9.88e-03   8.3 1.00e+00 3.91e-03h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 470  3.8790915e-01 1.31e-01 1.49e+06  -1.0 2.55e-02   7.8 1.00e+00 1.95e-03h 10
 471  3.8790907e-01 1.32e-01 1.48e+06  -1.0 1.24e-02   8.2 1.00e+00 3.91e-03h  9
 472  3.8790870e-01 1.34e-01 1.48e+06  -1.0 2.95e-02   7.8 1.00e+00 1.95e-03h 10
 473  3.8790852e-01 1.35e-01 1.48e+06  -1.0 1.46e-02   8.2 1.00e+00 3.91e-03h  9
 474  3.8790799e-01 1.36e-01 1.48e+06  -1.0 3.16e-02   7.7 1.00e+00 1.95e-03h 10
 475  3.8790785e-01 1.37e-01 1.48e+06  -1.0 1.87e-01   7.2 1.00e+00 1.22e-04h 14
 476  3.8790717e-01 1.38e-01 1.48e+06  -1.0 3.36e-02   7.7 1.00e+00 1.95e-03h 10
 477  3.8780413e-01 9.80e+01 2.72e+07  -1.0 1.75e-02   8.1 1.00e+00 1.00e+00w  1
 478  3.8748847e-01 2.52e+01 8.19e+06  -1.0 2.25e-02   7.6 1.00e+00 1.00e+00w  1
 479  3.8711181e-01 9.74e+00 9.97e+06  -1.0 4.53e-02   7.1 1.00e+00 1.00e+00w  1
iter    objective    inf_pr 

 561 -2.3171275e-01 3.46e+00 7.57e+02  -1.0 3.85e+00    -  6.05e-01 3.66e-01f  1
 562 -2.1690382e-01 1.85e+00 8.66e+02  -1.0 8.27e+00    -  3.62e-01 1.38e-01f  1
 563 -2.1608123e-01 1.83e+01 1.37e+03  -1.0 1.17e+01    -  2.20e-01 2.73e-01f  1
 564 -1.9883068e-01 2.49e+02 2.73e+03  -1.0 1.30e+01    -  1.90e-01 2.70e-01f  1
 565 -1.9475931e-01 5.20e+01 1.06e+06  -1.0 1.53e+00   4.1 4.05e-01 1.00e+00h  1
 566 -1.9562062e-01 3.89e+01 7.87e+05  -1.0 5.12e-01   3.6 4.81e-01 2.78e-01h  1
 567 -1.9619143e-01 3.33e+01 6.72e+05  -1.0 2.31e+00   3.1 2.59e-01 1.53e-01h  1
 568 -1.9789479e-01 2.44e+01 1.19e+06  -1.0 1.92e+00   3.5 4.06e-01 2.96e-01h  1
 569 -1.7170300e-01 4.89e+01 1.08e+08  -1.0 3.47e+00   4.0 3.62e-02 2.15e-01f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 570 -1.7195771e-01 1.63e+01 4.62e+07  -1.0 1.83e-01   6.2 4.81e-02 1.00e+00f  1
 571 -1.7538618e-01 5.88e+00 1.17e+07  -1.0 1.52e-01   5.7 1.00e+00 1.00e+00h  1
 572 -1.8678260e-01 2.68e+00

 653  2.0977979e-01 4.54e-02 7.25e+09  -1.0 1.23e+01    -  1.32e-02 3.10e-05h  1
 654r 2.0977979e-01 4.54e-02 1.00e+03  -1.0 0.00e+00    -  0.00e+00 3.19e-07R  5
 655r 2.0427060e-01 1.45e-03 1.30e+03  -1.0 2.06e+01    -  1.68e-01 2.19e-03f  1
 656  2.0479873e-01 1.45e-03 4.45e+05  -1.0 7.60e-01    -  9.66e-01 3.21e-04h  1
 657 -4.5477006e-01 7.12e-02 2.17e+05  -1.0 3.03e-01    -  5.11e-01 1.00e+00f  1
 658 -4.5038619e-01 7.10e-02 3.54e+07  -1.0 1.33e+00    -  3.90e-01 2.38e-03h  1
 659r-4.5038619e-01 7.10e-02 1.00e+03  -1.0 0.00e+00    -  0.00e+00 4.52e-07R  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 660r-4.3109749e-01 1.63e-02 1.58e+03  -1.0 5.04e+01    -  3.33e-01 1.40e-03f  1
 661 -4.2839363e-01 1.63e-02 6.95e+05  -1.0 1.31e+00    -  9.86e-01 1.49e-03h  1
 662 -6.4212818e-01 1.51e-02 3.37e+05  -1.0 1.94e-01    -  5.15e-01 1.00e+00f  1
 663 -5.2241660e-01 1.70e-02 2.26e+06  -1.0 1.67e+00    -  9.17e-03 6.25e-02f  5
 664  3.0479963e+00 1.56e-02

In [26]:
jac_init_1 = doe_object.analysis_square.jac_extracted.copy()
print(type(jac_init_1))
print(jac_init_1[0,0,0])

print(jac_init_1)

fo = open('jac_initial', 'wb')
pickle.dump(jac_init_1, fo)
fo.close



<class 'numpy.ndarray'>
-1.2203098649458607e-51
[[[-1.22030986e-51  1.36039977e-08  9.30013128e-09  6.92615755e-09
    4.82550554e-09  3.45281161e-09  2.36193238e-09  1.94081881e-09
    2.07622136e-09  3.07009918e-09  5.40779861e-09  9.37480479e-09
    1.66942383e-08  2.77556077e-08  4.71572907e-08  7.55335897e-08
    1.23975736e-07  1.94168463e-07  3.08533186e-07  4.76674786e-07
    7.27285370e-07  1.11080265e-06  1.61052899e-06  2.41876277e-06
    3.30287903e-06  4.84456231e-06  6.21686601e-06  8.84097764e-06
    1.07718918e-05  1.45441689e-05  1.73674147e-05  2.15211590e-05
    2.64912603e-05  2.91853734e-05  3.89168809e-05  3.68603619e-05
    5.52680495e-05  4.38549036e-05  7.61454347e-05  7.18529199e-05
    5.79409486e-05  4.41709623e-05  3.13020448e-05  1.97151710e-05
    9.57817181e-06  9.23101839e-07 -6.30206872e-06 -1.21986184e-05
   -1.68937014e-05 -2.05248744e-05 -2.32300912e-05 -2.51414722e-05
   -2.63816666e-05 -2.70619723e-05 -2.72816307e-05 -2.71278828e-05
   -2.66765041

<function BufferedWriter.close>

In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Heatmap

In [ ]:
#design_ranges = [list(np.linspace(293,333,5)), list(np.linspace(0.1,0.3,6))]
design_ranges = [[293, 303, 313, 323, 333], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

dv_apply_name = ['temp_feed','yfeed']

dv_apply_time = [[0],[0]]

# Define experiments
exp1 = generate_exp(313.15, 303.15, 0.15)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

print(design_ranges)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=False,
                                    scale_nominal_param_value=True, 
                                     store_name='20210919_heatmap', read_name='20210919_heatmap'
                                    )



In [ ]:
test = all_fim.extract_criteria()

fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','yfeed'], 'Fixed bed','$feed T [K]$', 'y' )

In [ ]:
na = ['temp_feed', 'yfeed']
dd = ['temp_feed', 'yfeed']
a = [[293.0, 303.0, 313.0, 323.0, 333.0], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

In [ ]:
print(a[na.index('temp_feed')])